In [1]:
import warnings
warnings.filterwarnings("ignore")

from nilearn import datasets # Fetch data using nilearn
adhd = datasets.fetch_adhd(n_subjects=10)          # ADHD200 preprocessed data (Athena pipeline)

In [2]:
adhd.func[0]

'/home/jovyan/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz'

In [3]:
# Load voxel-level data - a brain mask is automatically computed
from nilearn import plotting, input_data
masker_voxel = input_data.NiftiMasker(detrend=True, standardize=True, smoothing_fwhm=5).fit(adhd.func)
tseries_voxel = []
for mm in range(10):
    tseries_voxel.append(masker_voxel.transform(adhd.func[mm]))

In [4]:
for mm in range(10):
    print(f"Time series with shape {tseries_voxel[mm].shape} (# time points, # voxels))")

Time series with shape (176, 85878) (# time points, # voxels))
Time series with shape (176, 85878) (# time points, # voxels))
Time series with shape (176, 85878) (# time points, # voxels))
Time series with shape (175, 85878) (# time points, # voxels))
Time series with shape (77, 85878) (# time points, # voxels))
Time series with shape (77, 85878) (# time points, # voxels))
Time series with shape (261, 85878) (# time points, # voxels))
Time series with shape (260, 85878) (# time points, # voxels))
Time series with shape (260, 85878) (# time points, # voxels))
Time series with shape (261, 85878) (# time points, # voxels))


In [5]:
import mdf

In [7]:
K = 20
mdf.mdf(tseries_voxel,K)

99.99883555741874

In [8]:
ncores=32
memory_limit='64GB'
n_workers=11
processes=False
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(ncores=32,memory_limit='64GB',n_workers=11,processes=False)
client = Client(cluster)

In [9]:
JE = mdf.mdf_dask_tall(client,tseries_voxel,K)
JE.result()

99.99883555741873

In [ ]:
#from dask.distributed import Client

#client = Client("tcp://127.0.0.1:44955")
#client

In [ ]:
print(client)

In [ ]:
A = client.map(square, range(10))
B = client.map(neg, A)
total = client.submit(sum, B)
total.result()

In [ ]:
def square(x):
    return x ** 2

A = client.map(square, range(10))
A

In [ ]:
type(A)

In [ ]:
data_dict = {'a':1,'b':2}
data_dict['a']

In [ ]:
data_dict['a']